## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-10-11-44-30 +0000,bbc,Four arrested in connection with M&S and Co-op...,https://www.bbc.com/news/articles/cwykgrv374eo
1,2025-07-10-11-44-08 +0000,bbc,What is an ISA and how might the rules change?,https://www.bbc.com/news/articles/c0k7enxkxndo
2,2025-07-10-11-32-32 +0000,nyt,Trump Seems to Be Warming to What Europe Wants...,https://www.nytimes.com/2025/07/10/world/europ...
3,2025-07-10-11-30-07 +0000,bbc,Brazil vows to match US tariffs after Trump th...,https://www.bbc.com/news/articles/c62dvyv60z7o
4,2025-07-10-11-26-04 +0000,bbc,Open up places with air-con to help in heatwav...,https://www.bbc.com/news/articles/c20r53ex631o


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2321/2741963311.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
11,trump,20
17,russia,7
15,ukraine,6
138,tariff,5
16,new,5


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
2,2025-07-10-11-32-32 +0000,nyt,Trump Seems to Be Warming to What Europe Wants...,https://www.nytimes.com/2025/07/10/world/europ...,45
45,2025-07-10-01-17-00 +0000,wsj,A brewing debate inside the Federal Reserve ov...,https://www.wsj.com/economy/central-banking/wh...,44
60,2025-07-09-21-39-44 +0000,nyt,Russia Makes Record Attack on Ukraine as Trump...,https://www.nytimes.com/2025/07/09/world/europ...,44
82,2025-07-09-16-39-00 +0000,wsj,Businesses hoping for clarity on where Trump t...,https://www.wsj.com/economy/trade/trump-tariff...,43
48,2025-07-10-00-26-49 +0000,nyt,"Trump Pledges 50% Tariffs Against Brazil, Citi...",https://www.nytimes.com/2025/07/09/world/ameri...,39


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
2,45,2025-07-10-11-32-32 +0000,nyt,Trump Seems to Be Warming to What Europe Wants...,https://www.nytimes.com/2025/07/10/world/europ...
45,24,2025-07-10-01-17-00 +0000,wsj,A brewing debate inside the Federal Reserve ov...,https://www.wsj.com/economy/central-banking/wh...
64,22,2025-07-09-20-25-00 +0000,wsj,Minutes from the Fed’s June policy meeting tea...,https://www.wsj.com/economy/central-banking/ju...
20,21,2025-07-10-09-35-06 +0000,nyt,UK and France Agree to First Ever Nuclear Weap...,https://www.nytimes.com/2025/07/10/world/europ...
93,19,2025-07-09-13-51-41 +0000,bbc,French police raid on National Rally HQ prompt...,https://www.bbc.com/news/articles/ckg5kd04e1jo
63,19,2025-07-09-20-36-58 +0000,bbc,'Huge wall of water': Texas man stood on meter...,https://www.bbc.com/news/articles/c3358pz41d6o
70,16,2025-07-09-19-47-41 +0000,nyt,Can Canada Leverage Its Unique U.S. Relationsh...,https://www.nytimes.com/2025/07/09/world/canad...
82,15,2025-07-09-16-39-00 +0000,wsj,Businesses hoping for clarity on where Trump t...,https://www.wsj.com/economy/trade/trump-tariff...
48,15,2025-07-10-00-26-49 +0000,nyt,"Trump Pledges 50% Tariffs Against Brazil, Citi...",https://www.nytimes.com/2025/07/09/world/ameri...
36,15,2025-07-10-05-11-58 +0000,bbc,North Sea operators 'running out of time' to p...,https://www.bbc.com/news/articles/cn4101grdxvo


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
